# Basic Usage of Multi-Subset InstABoost

This notebook demonstrates the basic usage of the multi-subset InstABoost implementation.

We'll:
1. Load a small transformer model (GPT-2)
2. Create a simple instruction + question prompt
3. Apply boosting to different parts with different bias values
4. Compare outputs with and without boosting

In [1]:
# Add parent directory to path to import src modules
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # Must be set before importing torch
os.environ["USE_TORCH"] = "1"  # Force Hugging Face to use PyTorch
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppress TensorFlow logging
os.environ["TRANSFORMERS_NO_TF"] = "1"  # Disable TensorFlow in transformers

import sys
sys.path.append('..')

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.boost_config import TokenSubset, BoostConfig
from src.token_utils import create_token_subset_from_substring
from src.attention_hook import register_boost_hooks, unregister_boost_hooks, update_bias_mask

# Force CPU to avoid MPS hanging issues on macOS
device = torch.device('cpu')

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

PyTorch version: 2.7.1
Device: cpu


## 1. Load Model and Tokenizer

We'll start with GPT-2 small for quick testing.

In [2]:
# Load model and tokenizer
model_name = "gpt2"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Use CPU device (defined in cell-1 to avoid MPS issues)
model = model.to(device)

print(f"Model loaded on {device}")
print(f"Model type: {model.config.model_type}")
print(f"Number of layers: {model.config.n_layer}")

Loading gpt2...
Model loaded on cpu
Model type: gpt2
Number of layers: 12


## 2. Prepare Input

Create a prompt with an instruction and a question.

In [3]:
# Create input text
text = "Instruction: Answer in French. Question: What is the capital of France?"

print(f"Input text: {text}")
print(f"\nTokenization:")

# Tokenize
tokens = tokenizer.encode(text, add_special_tokens=False)
for i, token_id in enumerate(tokens):
    token_text = tokenizer.decode([token_id])
    print(f"  {i}: '{token_text}' (id={token_id})")

Input text: Instruction: Answer in French. Question: What is the capital of France?

Tokenization:
  0: 'Inst' (id=6310)
  1: 'ruction' (id=2762)
  2: ':' (id=25)
  3: ' Answer' (id=23998)
  4: ' in' (id=287)
  5: ' French' (id=4141)
  6: '.' (id=13)
  7: ' Question' (id=18233)
  8: ':' (id=25)
  9: ' What' (id=1867)
  10: ' is' (id=318)
  11: ' the' (id=262)
  12: ' capital' (id=3139)
  13: ' of' (id=286)
  14: ' France' (id=4881)
  15: '?' (id=30)


## 3. Generate Without Boosting (Baseline)

In [4]:
# Tokenize input
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

# Generate without boosting
print("Generating without boosting...")
with torch.no_grad():
    output_baseline = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 30,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

baseline_text = tokenizer.decode(output_baseline[0], skip_special_tokens=True)
print(f"\nBaseline output:\n{baseline_text}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating without boosting...

Baseline output:
Instruction: Answer in French. Question: What is the capital of France? Answer: Ville, Paris, France.

Q. What is Paris? Answer: The capital of France.

Q. What is


## 4. Create Boost Configuration

Define which parts of the input to boost and with what bias values.

In [5]:
# Create token subsets from substrings
instruction_subset = create_token_subset_from_substring(
    name="instruction",
    text=text,
    substring="Instruction: Answer in French.",
    tokenizer=tokenizer,
    bias=2.0  # Strong boost for instruction
)

question_subset = create_token_subset_from_substring(
    name="question",
    text=text,
    substring="Question: What is the capital of France?",
    tokenizer=tokenizer,
    bias=1.0  # Moderate boost for question
)

print(f"Instruction subset: {instruction_subset}")
print(f"Question subset: {question_subset}")

# Create boost configuration
config = BoostConfig(subsets=[instruction_subset, question_subset])
print(f"\nBoost config: {config}")

Instruction subset: TokenSubset(name='instruction', indices=[0, 1, 2]..., bias=2.0)
Question subset: TokenSubset(name='question', indices=[7, 8, 9]..., bias=1.0)

Boost config: BoostConfig(2 subsets, all layers, all heads, combination='sum')


## 5. Generate With Boosting

In [6]:
# Register hooks
print("Registering boost hooks...")
handle = register_boost_hooks(model, config, input_length=input_ids.shape[1])

# Update bias mask with actual sequence length
update_bias_mask(handle, seq_length=input_ids.shape[1], device=device)

print(f"Bias mask: {handle.bias_mask}")

# Generate with boosting
print("\nGenerating with boosting...")
with torch.no_grad():
    output_boosted = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 30,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

boosted_text = tokenizer.decode(output_boosted[0], skip_special_tokens=True)
print(f"\nBoosted output:\n{boosted_text}")

# Clean up hooks
unregister_boost_hooks(handle)
print("\nHooks unregistered.")

Registering boost hooks...
Bias mask: tensor([2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Generating with boosting...

Boosted output:
Instruction: Answer in French. Question: What is the capital of France? Answer: La City. Question: What is the capital of France? Answer: La Nationale. Question: What is the capital of France? Answer

Hooks unregistered.


## 6. Compare Outputs

In [7]:
print("=" * 80)
print("COMPARISON")
print("=" * 80)
print(f"\nInput:\n{text}")
print(f"\n{'Baseline (no boosting):':-^80}")
print(baseline_text)
print(f"\n{'Boosted (instruction=2.0, question=1.0):':-^80}")
print(boosted_text)
print("=" * 80)

COMPARISON

Input:
Instruction: Answer in French. Question: What is the capital of France?

----------------------------Baseline (no boosting):-----------------------------
Instruction: Answer in French. Question: What is the capital of France? Answer: Ville, Paris, France.

Q. What is Paris? Answer: The capital of France.

Q. What is

--------------------Boosted (instruction=2.0, question=1.0):--------------------
Instruction: Answer in French. Question: What is the capital of France? Answer: La City. Question: What is the capital of France? Answer: La Nationale. Question: What is the capital of France? Answer


## 7. Test with Different Bias Values

Try different bias configurations to see their effects.

In [8]:
# Test different bias values
bias_values = [
    (0.5, 0.5),   # Weak boosting
    (1.0, 0.5),   # Moderate instruction boost
    (3.0, 1.0),   # Strong instruction boost
    (5.0, 2.0),   # Very strong boosting
]

print("Testing different bias values...\n")

for inst_bias, quest_bias in bias_values:
    # Create new config
    inst_subset = create_token_subset_from_substring(
        "instruction", text, "Instruction: Answer in French.",
        tokenizer, inst_bias
    )
    quest_subset = create_token_subset_from_substring(
        "question", text, "Question: What is the capital of France?",
        tokenizer, quest_bias
    )
    config = BoostConfig(subsets=[inst_subset, quest_subset])

    # Register hooks
    handle = register_boost_hooks(model, config, input_length=input_ids.shape[1])
    update_bias_mask(handle, seq_length=input_ids.shape[1], device=device)

    # Generate
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=input_ids.shape[1] + 30,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # Clean up
    unregister_boost_hooks(handle)

    print(f"Bias (inst={inst_bias}, quest={quest_bias}):")
    print(f"{result}\n")
    print("-" * 80)

Testing different bias values...

Bias (inst=0.5, quest=0.5):
Instruction: Answer in French. Question: What is the capital of France? Answer: France. Question: What is the name of the city in France? Answer: Paris. Question: What is the name of the country in

--------------------------------------------------------------------------------
Bias (inst=1.0, quest=0.5):
Instruction: Answer in French. Question: What is the capital of France? Answer: The capital of France, and France, is France. Question: What is the name of the city in France as of August 31, 1862

--------------------------------------------------------------------------------
Bias (inst=3.0, quest=1.0):
Instruction: Answer in French. Question: What is the capital of France? Answer: Fondation, Fondation, Fondation, Fondation, Fondation. Question: What is the capital of France

--------------------------------------------------------------------------------
Bias (inst=5.0, quest=2.0):
Instruction: Answer in French. Questio

## Summary

This notebook demonstrated:
1. Loading a model and tokenizer
2. Creating token subsets from substrings
3. Configuring multi-subset boosting
4. Applying boosting during generation
5. Comparing outputs with different bias values

Next steps:
- Try with larger models (Mistral-7B-Instruct)
- Test with more complex prompts
- Analyze attention patterns